In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
import numpy as np
import matplotlib.pyplot as plt
from feature_manager import FeatureManager
from classifier.dnn_classifier import DNNClassifer
from random import randint
from keras import callbacks
from visualizer import *


2023-01-17 23:40:54.446082: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-01-17 23:40:54.446183: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-01-17 23:40:54.446195: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.


In [2]:

data = pd.read_csv("../data/BTCUSDT-1d.csv", parse_dates=["Open Time"], index_col = "Open Time")
fm = FeatureManager()
fm.build_feature(data=data,lags=5)

In [3]:
classifier = DNNClassifer()
classifier.configure(hl=2, hu = 100, dropout=True, input_dim=len(fm.cols))
classifier.prepare_data(data = fm.df,cols = fm.cols,target_col = "dir", random_state=randint(0,100))
classifier.run()

2023-01-17 23:40:58.252257: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2023-01-17 23:40:58.252336: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: andy-GA-970A-D3
2023-01-17 23:40:58.252362: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: andy-GA-970A-D3
2023-01-17 23:40:58.252612: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 520.61.5
2023-01-17 23:40:58.252706: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 520.61.5
2023-01-17 23:40:58.252737: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 520.61.5


Epoch 1/100

Epoch 1: val_loss improved from inf to 0.70721, saving model to ../data/model_dnn_checkpoint.h5
40/40 - 1s - loss: 0.7835 - accuracy: 0.4716 - val_loss: 0.7072 - val_accuracy: 0.4834 - 1s/epoch - 35ms/step
Epoch 2/100

Epoch 2: val_loss improved from 0.70721 to 0.70475, saving model to ../data/model_dnn_checkpoint.h5
40/40 - 0s - loss: 0.7231 - accuracy: 0.5348 - val_loss: 0.7048 - val_accuracy: 0.4760 - 195ms/epoch - 5ms/step
Epoch 3/100

Epoch 3: val_loss improved from 0.70475 to 0.70302, saving model to ../data/model_dnn_checkpoint.h5
40/40 - 0s - loss: 0.7235 - accuracy: 0.5205 - val_loss: 0.7030 - val_accuracy: 0.4871 - 238ms/epoch - 6ms/step
Epoch 4/100

Epoch 4: val_loss improved from 0.70302 to 0.70082, saving model to ../data/model_dnn_checkpoint.h5
40/40 - 0s - loss: 0.7319 - accuracy: 0.5111 - val_loss: 0.7008 - val_accuracy: 0.4945 - 237ms/epoch - 6ms/step
Epoch 5/100

Epoch 5: val_loss improved from 0.70082 to 0.70039, saving model to ../data/model_dnn_checkpo

(0.5434782608695652, 0.3382352941176471)

In [ ]:
acc_list = []
cov_list = []
laps = 5

for i in range (0,laps):
    print("\n======= Lap {} =======".format(i+1))
    callbacks.backend.clear_session()
    classifier = DNNClassifer(neg_cutoff=0.40,pos_cutoff=0.6,epochs=50,train_size=0.7,val_size=0.15)
    classifier.configure(hl=2, hu = 100, dropout=True, input_dim=len(fm.cols))
    classifier.prepare_data(data = fm.df,cols = fm.cols,target_col = "dir", shuffle=True, random_state=i)
    accurracy, coverage = classifier.run()
    classifier.visualize_loss()
    classifier.visualize_accuracy()
    acc_list.append(accurracy)
    cov_list.append(coverage)

acc_arr = np.array(acc_list)
cov_arr = np.array(cov_list)

print("\n======")
print("Accuracy mean: {}, std: {}".format(acc_arr.mean(),acc_arr.std()))
print("Coverage mean: {}, std: {}".format(cov_arr.mean(),cov_arr.std()))
print("======")